In [4]:
library('dplyr')
library("zoo")
library(corrplot)
# library("Hmisc")

In [5]:
directory = '/media/nicolas/Elements/data/'

In [6]:
var <- list('DD','FG','FX','HU','PP','QQ','RR','SD','SS','TG','TN','TX','CC')

In [29]:
savecorr <- function(ID,var){
    
##            INIT          ##
    
    mydata <- NULL
    datelist <- NULL
    Data <- NULL
    mcor<- NULL
    path_to_data <- NULL
    j=0

## Création d'un dataframe avec des dates en rownames et des données journalières en colones
## Une colone par variable

    ## Une itération = Une cvariable = Une colone
    for (i in 1:length(var)){        
        path_to_data <- paste(directory,var[i],'/',var[i],'_STAID',sep='')
        
    ## Vérification de l'éxistence du fichier    
        if (!file.exists(paste(path_to_data,ID,'.txt',sep=''))){
            stop("Invalid ID")
        }
        
    ## Ouverture des donnée pour une station, une variable, on garde que la periode qui
    ## nous intéresse pour harmoniser la longueures des colones        
        Data <- read.table(paste(path_to_data,ID,'.txt',sep='')
                                  ,skip = 20,header=TRUE,sep=',')
        Data <- filter (Data,Data$DATE>="19700101" & Data$DATE<="20101231")
        
    ## Si il manque des données on arrète le programme 
        if (!length(Data$DATE)==14975){
            stop("Measurement period is invalid, change ID")
        }
        
    ## j compte le nombre de station qui passent les deux test ci-dessus
        j <- j+1    
        if (j==1 ){
        
        ## Première variable : on commence à construir le df et on garde une liste DATE de côté
            mydata=Data
            datelist <- Data$DATE       
            mydata <- mydata[,-c(1:3)]    #Supression des colones inutiles
        }
        else{
            
        ## Variables suivantes, on rajoute des colonnes à la suite du df 
            Data <- Data [,-c(1:3)]    #Supression des colones inutiles
            mydata <- cbind (mydata,Data)    
        } 
    }
    
    mydata$date <- datelist
    
    ## On vire les données de mauvaise qualité
    for ( i in 1:length(var)){
        mydata <- filter(mydata,mydata[,2*i]==0)
    }
    
    ## Suppression des colones " qualitéde la donnée " 
    mydata<-mydata[,-2*1:length(var)]
    
    # Date en row names ( obligé de le faire après le filtre qualité de donnée ?? )
    row.names(mydata)<-mydata$date
    mydata$date<-NULL
    
    # Tracée de la matrice de corrélation et enregistrement du plot en pdf ds ./test/
    mcor <- cor(mydata)
    pdf(file=paste('./test/',ID,'.pdf',sep=''))
    corrplot(mcor, type="upper", order="hclust", tl.col="black", tl.srt=45)
    dev.off()
    
    ## Décommenter les deu lignes ci dessous pour renvoyer ID 
    ## à décommenter si on veut utiliser savecorr2
    
    # kk<-ID
    # return(kk)
}


savecorr2 <- function (x,y) {
## Utiliser pourignorer les messages d'erreures ds savecorr 
## par ex pour avoir la liste des stations pour lesquels 
## la matrice de corrélation à pu être calculée
  out <- tryCatch(savecorr(x,y), error = function(e) NULL)
  return(out)
}


Wichsta <- function(variables_list){
## Renvoie une liste de character correspondant au ID de stations pour lesquels des données 
## sont disponibles dans tous les dossiers de variables_list
    
    # list_comm <-NULL
    for (i in 1:length(variables_list)){
        sources=read.csv(file =paste(directory,variables_list[i],"/stations.txt",sep=''),sep=",", 
                   skip = 17,header = TRUE)
        sources$STAID<-as.integer(sources$STAID)
        if (i==1){list_comm<-sources['STAID']}
        else{list_comm<-intersect(list_comm,sources['STAID'])}
        }
    list_comm$STAID<-formatC(list_comm$STAID, width = 6, format = "d", flag = "0")

    return(list_comm)
}

In [36]:
sta = Wichsta(var)

In [157]:
for ( i in 1:length(sta[[1]])){
    kk<-append(kk,try(savecorr2(sta[[1]][i],var), silent = TRUE))
}

kk

[1] "000120" "000123" "000190" "000193" "000265" "000266" "000268" "000328"
 [9] "000970" "001046" "001718" "002135" "002138" "002139" "002140" "002142"
[17] "002143" "002605" "002656" "002720"